# ResNet18 V2  [Train]
<br>
Code inspired by: https://www.kaggle.com/code/kadircandrisolu/efficientnet-b0-pytorch-inference-birdclef-25

In [1]:
!source activate cs336_data

### Importing

In [2]:
import os
import logging
import random
import gc
import time
import cv2
import math
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, multilabel_confusion_matrix
import librosa

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

from common import DATA_KOA_PATH, SPECTROGRAM_PATH, MODEL_OUTPUT_PATH


### Configuration

In [3]:
class CFG:
    seed = 42
    debug = False
    apex = False
    print_freq = 100
    num_workers = 2

    OUTPUT_DIR = str(MODEL_OUTPUT_PATH / "global_attention2")

    train_datadir = str(DATA_KOA_PATH/ "train_audio")
    train_csv = str(DATA_KOA_PATH/ "train.csv")
    taxonomy_csv = str(DATA_KOA_PATH/ "taxonomy.csv")
    spectrogram_npy = str(SPECTROGRAM_PATH / "birdclef2025_melspec_5sec_256_256.npy")

    in_channels = 1
    model_name = "resnet18"  

    LOAD_DATA = True
    FS = 32000
    TARGET_DURATION = 5.0
    TARGET_SHAPE = (256, 256)

    N_FFT = 1034
    HOP_LENGTH = 64
    N_MELS = 136
    FMIN = 20
    FMAX = 16000

    device = "cuda" if torch.cuda.is_available() else "cpu"
    epochs = 10
    batch_size = 32
    criterion = "BCEWithLogitsLoss"

    n_fold = 5
    selected_folds = [0, 1, 2, 3, 4]

    optimizer = "AdamW"
    lr = 5e-4
    weight_decay = 1e-5

    scheduler = "CosineAnnealingLR"
    min_lr = 1e-6
    T_max = epochs

    aug_prob = 0.5

    def update_debug_settings(self):
        if self.debug:
            self.epochs = 2
            self.selected_folds = [0]
cfg = CFG()


### Utility

In [4]:
# Utility 

def set_seed(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(cfg.seed)


### Preprocessing

In [5]:
# Audio → Mel helpers 

def audio2melspec(audio_data: np.ndarray, cfg: CFG):
    if np.isnan(audio_data).any():
        mean_signal = np.nanmean(audio_data)
        audio_data = np.nan_to_num(audio_data, nan=mean_signal)

    mel = librosa.feature.melspectrogram(
        y=audio_data,
        sr=cfg.FS,
        n_fft=cfg.N_FFT,
        hop_length=cfg.HOP_LENGTH,
        n_mels=cfg.N_MELS,
        fmin=cfg.FMIN,
        fmax=cfg.FMAX,
        power=2.0,
    )
    mel_db = librosa.power_to_db(mel, ref=np.max)
    mel_norm = (mel_db - mel_db.min()) / (mel_db.max() - mel_db.min() + 1e-8)
    return mel_norm


def process_audio_file(audio_path: str, cfg: CFG):  # → (H, W) float32
    try:
        audio_data, _ = librosa.load(audio_path, sr=cfg.FS)
        target_samples = int(cfg.TARGET_DURATION * cfg.FS)

        if len(audio_data) < target_samples:
            n_copy = math.ceil(target_samples / len(audio_data))
            audio_data = np.concatenate([audio_data] * n_copy)

        start = max(0, len(audio_data) // 2 - target_samples // 2)
        center_audio = audio_data[start : start + target_samples]
        if len(center_audio) < target_samples:
            center_audio = np.pad(center_audio, (0, target_samples - len(center_audio)))

        mel = audio2melspec(center_audio, cfg)
        if mel.shape != cfg.TARGET_SHAPE:
            mel = cv2.resize(mel, cfg.TARGET_SHAPE, interpolation=cv2.INTER_LINEAR)
        return mel.astype(np.float32)
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None

### Dataset Preprocessing and Data Augmentations

In [6]:
# Dataset 
class BirdCLEFDataset(Dataset):
    def __init__(self, df: pd.DataFrame, cfg: CFG, spectrograms=None, mode="train"):
        self.df = df.reset_index(drop=True)
        self.cfg = cfg
        self.mode = mode
        self.spectrograms = spectrograms

        tax = pd.read_csv(cfg.taxonomy_csv)
        self.labels = tax["primary_label"].tolist()
        self.num_classes = len(self.labels)
        self.label2idx = {l: i for i, l in enumerate(self.labels)}

        if "filepath" not in self.df.columns:
            self.df["filepath"] = cfg.train_datadir + "/" + self.df.filename
        if "samplename" not in self.df.columns:
            self.df["samplename"] = (
                self.df.filename.map(lambda x: x.split("/")[0] + "-" + x.split("/")[-1].split(".")[0])
            )
        if cfg.debug:
            self.df = self.df.sample(min(1000, len(self.df)), random_state=cfg.seed)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        samp = row.samplename
        spec = None
        if self.spectrograms is not None and samp in self.spectrograms:
            spec = self.spectrograms[samp]
        elif not self.cfg.LOAD_DATA:
            spec = process_audio_file(row.filepath, self.cfg)
        if spec is None:
            spec = np.zeros(self.cfg.TARGET_SHAPE, dtype=np.float32)
        spec = torch.tensor(spec).unsqueeze(0)  # (1, H, W)
        if self.mode == "train" and random.random() < self.cfg.aug_prob:
            spec = self.spec_augment(spec)
        target = self.encode(row.primary_label)
        return {"melspec": spec, "target": torch.tensor(target, dtype=torch.float32)}

    def encode(self, label):
        t = np.zeros(self.num_classes)
        if label in self.label2idx:
            t[self.label2idx[label]] = 1.0
        return t

    @staticmethod
    def spec_augment(spec):  # simple time/freq masking
        if random.random() < 0.5:
            w = random.randint(5, 20)
            start = random.randint(0, spec.shape[2] - w)
            spec[0, :, start : start + w] = 0
        if random.random() < 0.5:
            h = random.randint(5, 20)
            start = random.randint(0, spec.shape[1] - h)
            spec[0, start : start + h, :] = 0
        return spec


def collate_fn(batch):
    keys = batch[0].keys()
    collated = {k: [] for k in keys}
    for item in batch:
        for k in keys:
            collated[k].append(item[k])
    collated["melspec"] = torch.stack(collated["melspec"])
    collated["target"] = torch.stack(collated["target"])
    return collated

In [7]:
class BirdCLEFDatasetFromNPY(Dataset):
    def __init__(self, df, cfg, spectrograms=None, mode="train"):
        self.df = df
        self.cfg = cfg
        self.mode = mode

        self.spectrograms = spectrograms
        
        taxonomy_df = pd.read_csv(self.cfg.taxonomy_csv)
        self.species_ids = taxonomy_df['primary_label'].tolist()
        self.num_classes = len(self.species_ids)
        self.label_to_idx = {label: idx for idx, label in enumerate(self.species_ids)}

        if 'filepath' not in self.df.columns:
            self.df['filepath'] = self.cfg.train_datadir + '/' + self.df.filename
        
        if 'samplename' not in self.df.columns:
            self.df['samplename'] = self.df.filename.map(lambda x: x.split('/')[0] + '-' + x.split('/')[-1].split('.')[0])

        sample_names = set(self.df['samplename'])
        if self.spectrograms:
            found_samples = sum(1 for name in sample_names if name in self.spectrograms)
            print(f"Found {found_samples} matching spectrograms for {mode} dataset out of {len(self.df)} samples")
        
        if cfg.debug:
            self.df = self.df.sample(min(1000, len(self.df)), random_state=cfg.seed).reset_index(drop=True)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        samplename = row['samplename']
        spec = None

        if self.spectrograms and samplename in self.spectrograms:
            spec = self.spectrograms[samplename]
        elif not self.cfg.LOAD_DATA:
            spec = process_audio_file(row['filepath'], self.cfg)

        if spec is None:
            spec = np.zeros(self.cfg.TARGET_SHAPE, dtype=np.float32)
            if self.mode == "train":  # Only print warning during training
                print(f"Warning: Spectrogram for {samplename} not found and could not be generated")

        spec = torch.tensor(spec, dtype=torch.float32).unsqueeze(0)  # Add channel dimension

        if self.mode == "train" and random.random() < self.cfg.aug_prob:
            spec = self.apply_spec_augmentations(spec)
        
        target = self.encode_label(row['primary_label'])
        
        if 'secondary_labels' in row and row['secondary_labels'] not in [[''], None, np.nan]:
            if isinstance(row['secondary_labels'], str):
                secondary_labels = eval(row['secondary_labels'])
            else:
                secondary_labels = row['secondary_labels']
            
            for label in secondary_labels:
                if label in self.label_to_idx:
                    target[self.label_to_idx[label]] = 1.0
        
        return {
            'melspec': spec, 
            'target': torch.tensor(target, dtype=torch.float32),
            'filename': row['filename']
        }
    
    def apply_spec_augmentations(self, spec):
        """Apply augmentations to spectrogram"""
    
        # Time masking (horizontal stripes)
        if random.random() < 0.5:
            num_masks = random.randint(1, 3)
            for _ in range(num_masks):
                width = random.randint(5, 20)
                start = random.randint(0, spec.shape[2] - width)
                spec[0, :, start:start+width] = 0
        
        # Frequency masking (vertical stripes)
        if random.random() < 0.5:
            num_masks = random.randint(1, 3)
            for _ in range(num_masks):
                height = random.randint(5, 20)
                start = random.randint(0, spec.shape[1] - height)
                spec[0, start:start+height, :] = 0
        
        # Random brightness/contrast
        if random.random() < 0.5:
            gain = random.uniform(0.8, 1.2)
            bias = random.uniform(-0.1, 0.1)
            spec = spec * gain + bias
            spec = torch.clamp(spec, 0, 1) 
            
        return spec
    
    def encode_label(self, label):
        """Encode label to one-hot vector"""
        target = np.zeros(self.num_classes)
        if label in self.label_to_idx:
            target[self.label_to_idx[label]] = 1.0
        return target

### Model

In [8]:
# ResNet 18
class GlobalAttentionPool(nn.Module):
    """
    Learnable attention pooling:
    - Projects spatial features into keys and values
    - Uses a single learnable query to attend over H×W tokens
    """
    def __init__(self, input_dim, num_heads=8):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim=input_dim, num_heads=num_heads, batch_first=True)
        self.query = nn.Parameter(torch.randn(1, 1, input_dim))  # learnable query vector
        
    def forward(self, x):
        """
        Args:
            x: Tensor of shape (batch_size, seq_len, input_dim)
        Returns:
            out: Tensor of shape (batch_size, input_dim)
        """
        batch_size = x.size(0)

        # Expand the query for the batch
        query = self.query.expand(batch_size, -1, -1)  # [batch_size, 1, input_dim]

        # Attention
        attn_output, _ = self.attention(query=query, key=x, value=x)  # Output shape: (batch_size, 1, input_dim)

        return attn_output.squeeze(1)  # Remove sequence dimension → shape (batch_size, input_dim)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes, in_channels=1):
        super().__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(in_channels, 64, 3, stride=1, padding=1, bias=False)  # 3×3 stem (no maxpool)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.pool = nn.AdaptiveAvgPool2d((4, 4))  # Reduce to 4x4 features first (optional)
        self.attention_pool = GlobalAttentionPool(input_dim=512 * block.expansion)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        self._init_weights()

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for s in strides:
            layers.append(block(self.in_planes, planes, s))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, a=math.sqrt(5))
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        out = self.bn1(self.conv1(x))
        out = F.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.pool(out)              # (B, C, 4, 4)
        out = out.flatten(2)              # (B, C, 16)
        out = out.transpose(1, 2)          # (B, 16, C)
        out = self.attention_pool(out)     # (B, C)
        out = self.fc(out)                 # (B, num_classes)
        return out



def resnet18_spectrogram(num_classes: int, in_channels: int = 1):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes, in_channels)

### Training Utilities

In [9]:
def get_optimizer(model, cfg):
  
    if cfg.optimizer == 'Adam':
        optimizer = optim.Adam(
            model.parameters(),
            lr=cfg.lr,
            weight_decay=cfg.weight_decay
        )
    elif cfg.optimizer == 'AdamW':
        optimizer = optim.AdamW(
            model.parameters(),
            lr=cfg.lr,
            weight_decay=cfg.weight_decay
        )
    elif cfg.optimizer == 'SGD':
        optimizer = optim.SGD(
            model.parameters(),
            lr=cfg.lr,
            momentum=0.9,
            weight_decay=cfg.weight_decay
        )
    else:
        raise NotImplementedError(f"Optimizer {cfg.optimizer} not implemented")
        
    return optimizer

def get_scheduler(optimizer, cfg):
   
    if cfg.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=cfg.T_max,
            eta_min=cfg.min_lr
        )
    elif cfg.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.5,
            patience=2,
            min_lr=cfg.min_lr,
            verbose=True
        )
    elif cfg.scheduler == 'StepLR':
        scheduler = lr_scheduler.StepLR(
            optimizer,
            step_size=cfg.epochs // 3,
            gamma=0.5
        )
    elif cfg.scheduler == 'OneCycleLR':
        scheduler = None  
    else:
        scheduler = None
        
    return scheduler

def get_criterion(cfg):
 
    if cfg.criterion == 'BCEWithLogitsLoss':
        criterion = nn.BCEWithLogitsLoss()
    else:
        raise NotImplementedError(f"Criterion {cfg.criterion} not implemented")
        
    return criterion

### Training Loop and Training

In [10]:
def train_one_epoch(model, loader, optimizer, criterion, device, scheduler=None):
    
    model.train()
    losses = []
    all_targets = []
    all_outputs = []
    
    pbar = tqdm(enumerate(loader), total=len(loader), desc="Training")
    
    for step, batch in pbar:
    
        if isinstance(batch['melspec'], list):
            batch_outputs = []
            batch_losses = []
            
            for i in range(len(batch['melspec'])):
                inputs = batch['melspec'][i].unsqueeze(0).to(device)
                target = batch['target'][i].unsqueeze(0).to(device)
                
                optimizer.zero_grad()
                output = model(inputs)
                loss = criterion(output, target)
                loss.backward()
                
                batch_outputs.append(output.detach().cpu())
                batch_losses.append(loss.item())
            
            optimizer.step()
            outputs = torch.cat(batch_outputs, dim=0).numpy()
            loss = np.mean(batch_losses)
            targets = batch['target'].numpy()
            
        else:
            inputs = batch['melspec'].to(device)
            targets = batch['target'].to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            
            if isinstance(outputs, tuple):
                outputs, loss = outputs  
            else:
                loss = criterion(outputs, targets)
                
            loss.backward()
            optimizer.step()
            
            outputs = outputs.detach().cpu().numpy()
            targets = targets.detach().cpu().numpy()
        
        if scheduler is not None and isinstance(scheduler, lr_scheduler.OneCycleLR):
            scheduler.step()
            
        all_outputs.append(outputs)
        all_targets.append(targets)
        losses.append(loss if isinstance(loss, float) else loss.item())
        pbar.set_postfix({
            "train_loss": np.mean(losses[-10:]) if losses else 0,
            "lr": optimizer.param_groups[0]["lr"],
        })
    
    all_outputs = np.concatenate(all_outputs)
    all_targets = np.concatenate(all_targets)
    auc = calculate_auc(all_targets, all_outputs)
    extra = calculate_extra_metrics(all_targets, all_outputs)
    avg_loss = np.mean(losses)



    
    return avg_loss, auc, extra

def validate(model, loader, criterion, device):
   
    model.eval()
    losses = []
    all_targets = []
    all_outputs = []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation"):
            if isinstance(batch['melspec'], list):
                batch_outputs = []
                batch_losses = []
                
                for i in range(len(batch['melspec'])):
                    inputs = batch['melspec'][i].unsqueeze(0).to(device)
                    target = batch['target'][i].unsqueeze(0).to(device)
                    
                    output = model(inputs)
                    loss = criterion(output, target)
                    
                    batch_outputs.append(output.detach().cpu())
                    batch_losses.append(loss.item())
                
                outputs = torch.cat(batch_outputs, dim=0).numpy()
                loss = np.mean(batch_losses)
                targets = batch['target'].numpy()
                
            else:
                inputs = batch['melspec'].to(device)
                targets = batch['target'].to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                
                outputs = outputs.detach().cpu().numpy()
                targets = targets.detach().cpu().numpy()
            
            all_outputs.append(outputs)
            all_targets.append(targets)
            losses.append(loss if isinstance(loss, float) else loss.item())
    
    all_outputs = np.concatenate(all_outputs)
    all_targets = np.concatenate(all_targets)
    
    auc = calculate_auc(all_targets, all_outputs)
    avg_loss = np.mean(losses)
    extra = calculate_extra_metrics(all_targets, all_outputs)

    
    return avg_loss, auc, extra

def calculate_auc(targets, outputs):
  
    num_classes = targets.shape[1]
    aucs = []
    
    probs = 1 / (1 + np.exp(-outputs))
    
    for i in range(num_classes):
        
        if np.sum(targets[:, i]) > 0:
            class_auc = roc_auc_score(targets[:, i], probs[:, i])
            aucs.append(class_auc)
    
    return np.mean(aucs) if aucs else 0.0


def calculate_extra_metrics(targets, logits, thresh=0.09):
    """
    Compute macro-averaged F1, per-label accuracy and a multilabel
    confusion matrix given (N, C) targets and logits.
    """
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs >= thresh).astype(int)

    macro_f1 = f1_score(targets, preds, average="macro", zero_division=0)
    micro_f1 = f1_score(targets, preds, average="micro", zero_division=0)
    correct = (preds == targets).astype(int)
    label_acc = correct.mean(axis=0)          # vector length C
    mean_label_acc = float(label_acc.mean())  # single number for logging

    conf_mat = multilabel_confusion_matrix(targets, preds)

    return {
        "macro_f1":       float(macro_f1),
        "micro_f1":       float(micro_f1),
        "label_acc_mean": mean_label_acc,
        "label_acc_vec":  label_acc,   # keep full vector for later analysis
        "conf_matrix":    conf_mat,    # Tensor/ndarray
    }


In [11]:
def run_training(df, cfg):
    """Training function that can either use pre-computed spectrograms or generate them on-the-fly"""

    taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
    species_ids = taxonomy_df['primary_label'].tolist()
    cfg.num_classes = len(species_ids)
    out_dir = Path(cfg.OUTPUT_DIR)
    out_dir.mkdir(parents=True, exist_ok=True)

    if cfg.debug:
        cfg.update_debug_settings()

    spectrograms = None
    if cfg.LOAD_DATA:
        print("Loading pre-computed mel spectrograms from NPY file...")
        try:
            spectrograms = np.load(cfg.spectrogram_npy, allow_pickle=True).item()
            print(f"Loaded {len(spectrograms)} pre-computed mel spectrograms")
        except Exception as e:
            print(f"Error loading pre-computed spectrograms: {e}")
            print("Will generate spectrograms on-the-fly instead.")
            cfg.LOAD_DATA = False
    
    if not cfg.LOAD_DATA:
        print("Will generate spectrograms on-the-fly during training.")
        if 'filepath' not in df.columns:
            df['filepath'] = cfg.train_datadir + '/' + df.filename
        if 'samplename' not in df.columns:
            df['samplename'] = df.filename.map(lambda x: x.split('/')[0] + '-' + x.split('/')[-1].split('.')[0])
        
    skf = StratifiedKFold(n_splits=cfg.n_fold, shuffle=True, random_state=cfg.seed)
    
    best_scores = []

    
    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['primary_label'])):
        if fold not in cfg.selected_folds:
            continue
        metrics_log = [] 
            
        print(f'\n{"="*30} Fold {fold} {"="*30}')
        train_df = df.iloc[train_idx].reset_index(drop=True)
        val_df = df.iloc[val_idx].reset_index(drop=True)
        
        print(f'Training set: {len(train_df)} samples')
        print(f'Validation set: {len(val_df)} samples')
        
        train_dataset = BirdCLEFDatasetFromNPY(train_df, cfg, spectrograms=spectrograms, mode='train')
        val_dataset = BirdCLEFDatasetFromNPY(val_df, cfg, spectrograms=spectrograms, mode='valid')
        
        train_loader = DataLoader(
            train_dataset, 
            batch_size=cfg.batch_size, 
            shuffle=True, 
            num_workers=cfg.num_workers,
            pin_memory=True,
            collate_fn=collate_fn,
            drop_last=True
        )
        
        val_loader = DataLoader(
            val_dataset, 
            batch_size=cfg.batch_size, 
            shuffle=False, 
            num_workers=cfg.num_workers,
            pin_memory=True,
            collate_fn=collate_fn
        )
        
        model = resnet18_spectrogram(num_classes=cfg.num_classes, in_channels=cfg.in_channels).to(cfg.device)


        optimizer = get_optimizer(model, cfg)
        criterion = get_criterion(cfg)
        
        if cfg.scheduler == 'OneCycleLR':
            scheduler = lr_scheduler.OneCycleLR(
                optimizer,
                max_lr=cfg.lr,
                steps_per_epoch=len(train_loader),
                epochs=cfg.epochs,
                pct_start=0.1
            )
        else:
            scheduler = get_scheduler(optimizer, cfg)
        
        best_auc = 0
        best_epoch = 0
        
        for epoch in range(cfg.epochs):
            print(f"\nEpoch {epoch+1}/{cfg.epochs}")
            
            train_loss, train_auc, train_extra = train_one_epoch(
                model, 
                train_loader, 
                optimizer, 
                criterion, 
                cfg.device,
                scheduler if isinstance(scheduler, lr_scheduler.OneCycleLR) else None
            )
            
            val_loss, val_auc, val_extra = validate(model, val_loader, criterion, cfg.device)


            if scheduler is not None and not isinstance(scheduler, lr_scheduler.OneCycleLR):
                if isinstance(scheduler, lr_scheduler.ReduceLROnPlateau):
                    scheduler.step(val_loss)
                else:
                    scheduler.step()

            print(
                f"[Epoch {epoch+1}] "
                f"Train: loss {train_loss:.4f} | "
                f"AUC {train_auc:.4f} | "
                f"Macro-F1 {train_extra['macro_f1']:.4f} | "
                f"Micro-F1 {train_extra['micro_f1']:.4f} | "
                f"L-Acc {train_extra['label_acc_mean']:.4f}   ||   "
                f"Val: loss {val_loss:.4f} | "
                f"AUC {val_auc:.4f} | "
                f"Macro-F1 {val_extra['macro_f1']:.4f} | "
                f"Micro-F1 {val_extra['micro_f1']:.4f} | "
                f"L-Acc {val_extra['label_acc_mean']:.4f}"
            )

            metrics_log.append({
                "fold":         fold,
                "epoch":        epoch + 1,

                "train_loss":   train_loss,
                "train_auc":    train_auc,
                "train_macro_f1":   train_extra["macro_f1"],
                "train_micro_f1":   train_extra["micro_f1"],
                "train_lacc":   train_extra["label_acc_mean"],

                "val_loss":     val_loss,
                "val_auc":      val_auc,
                "val_macro_f1":     val_extra["macro_f1"],
                "val_micro_f1":     val_extra["micro_f1"],
                "val_lacc":     val_extra["label_acc_mean"],
            })


            
            if val_auc > best_auc:
                best_auc = val_auc
                best_epoch = epoch + 1
                print(f"New best AUC: {best_auc:.4f} at epoch {best_epoch}")
                np.save(out_dir / f"confmat_fold{fold}_best.npy",val_extra["conf_matrix"])
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
                    'epoch': epoch,
                    'val_auc': val_auc,
                    'train_auc': train_auc,
                    'cfg': vars(cfg)
                },  out_dir / f"model_fold{fold}_best.pth")
       
        pd.DataFrame(metrics_log).to_csv(
            out_dir / f"metrics_fold{fold}.csv",
            index=False
        )

        print(f"Saved per-epoch metrics → metrics_fold{fold}.csv")

        
        best_scores.append(best_auc)
        print(f"\nBest AUC for fold {fold}: {best_auc:.4f} at epoch {best_epoch}")
        
        # Clear memory
        del model, optimizer, scheduler, train_loader, val_loader
        torch.cuda.empty_cache()
        gc.collect()
    
    print("\n" + "="*60)
    print("Cross-Validation Results:")
    for fold, score in enumerate(best_scores):
        print(f"Fold {cfg.selected_folds[fold]}: {score:.4f}")
    print(f"Mean AUC: {np.mean(best_scores):.4f}")
    print("="*60)

In [12]:
if __name__ == "__main__":

    print("\nLoading training data...")
    train_df = pd.read_csv(cfg.train_csv)
    taxonomy_df = pd.read_csv(cfg.taxonomy_csv)

    print("\nStarting training...")
    print(f"LOAD_DATA is set to {cfg.LOAD_DATA}")
    if cfg.LOAD_DATA:
        print("Using pre-computed mel spectrograms from NPY file")
    else:
        print("Will generate spectrograms on-the-fly during training")
    
    run_training(train_df, cfg)
    
    print("\nTraining complete!")


Loading training data...

Starting training...
LOAD_DATA is set to True
Using pre-computed mel spectrograms from NPY file
Loading pre-computed mel spectrograms from NPY file...
Loaded 28564 pre-computed mel spectrograms

============================== Fold 0 ==============================
Training set: 22851 samples
Validation set: 5713 samples
Found 22851 matching spectrograms for train dataset out of 22851 samples
Found 5713 matching spectrograms for valid dataset out of 5713 samples

Epoch 1/10


Training:   0%|          | 0/714 [00:00<?, ?it/s]

Validation:   0%|          | 0/179 [00:00<?, ?it/s]

[Epoch 1] Train: loss 0.0348 | AUC 0.4976 | Macro-F1 0.0059 | Micro-F1 0.0140 | L-Acc 0.9873   ||   Val: loss 0.0308 | AUC 0.6046 | Macro-F1 0.0030 | Micro-F1 0.0243 | L-Acc 0.9939
New best AUC: 0.6046 at epoch 1

Epoch 2/10


Training:   0%|          | 0/714 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/179 [00:00<?, ?it/s]

[Epoch 9] Train: loss 0.0167 | AUC 0.9507 | Macro-F1 0.3557 | Micro-F1 0.4358 | L-Acc 0.9917   ||   Val: loss 0.0181 | AUC 0.9250 | Macro-F1 0.3189 | Micro-F1 0.4293 | L-Acc 0.9919
New best AUC: 0.9250 at epoch 9

Epoch 10/10


Training:   0%|          | 0/714 [00:00<?, ?it/s]

Validation:   0%|          | 0/179 [00:00<?, ?it/s]

[Epoch 10] Train: loss 0.0161 | AUC 0.9550 | Macro-F1 0.3886 | Micro-F1 0.4510 | L-Acc 0.9919   ||   Val: loss 0.0178 | AUC 0.9269 | Macro-F1 0.3149 | Micro-F1 0.4190 | L-Acc 0.9912
New best AUC: 0.9269 at epoch 10
Saved per-epoch metrics → metrics_fold4.csv

Best AUC for fold 4: 0.9269 at epoch 10

Cross-Validation Results:
Fold 0: 0.9229
Fold 1: 0.9244
Fold 2: 0.9270
Fold 3: 0.9254
Fold 4: 0.9269
Mean AUC: 0.9253

Training complete!
